## Prepare Dataset and Config Files

In [3]:
import glob


# download dataset
!wget https: // github.com/abdulazeezoj/omdena-abuja-anpd/raw/main/yolov4.zip

# unzip out dataset
!unzip yolov4.zip

# make train.txt and test.txt file
file_train = open('/content/yolov4/data/train.txt', 'w')
file_test = open('/content/yolov4/data/test.txt', 'w')

img_list = glob.glob(f"/content/yolov4/data/obj/*.jpg", recursive=False)
dataset_size = len(img_list)
train_pct = 0.9

train_idx = round(dataset_size * train_pct)
print(f"{len(img_list)} images found")
for i, img in enumerate(img_list, 1):
    file_train.write(img.replace("\\", "/") + "\n")
    
    if i > train_idx:
        file_test.write(img.replace("\\", "/") + "\n")

file_train.close()
file_test.close()

# cd into yolov4 directory
%cd yolov4


60 images found


## Train ANPD with YOLOv4

In [ ]:
# clone darknet repository
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# build darknet
%cd darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/OPENMP=0/OPENMP=1/' Makefile

!make


In [ ]:
# download initial weight

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# start training

!./darknet detector train /content/yolov4/data/obj.data /content/yolov4/data/obj.cfg /content/yolov4/darknet/yolov4.conv.137 -dont_show -map

In [ ]:
# resuming training [optional]

!./darknet detector train /content/yolov4/data/obj.data /content/yolov4/data/obj.cfg /content/yolov4/outputs/yolov4-obj_last.weights -dont_show -map

In [ ]:
# checking mAP

!./darknet detector map /content/yolov4/data/obj.data /content/yolov4/data/obj.cfg /content/yolov4/outputs/yolov4-obj_1000.weights

In [ ]:
# testing the ANPD model on image

!./darknet detector test /content/yolov4/data/obj.data /content/yolov4/data/obj.cfg /content/yolov4/outputs/yolov4-obj_last.weights /content/yolov4/test/img1.jpg -thresh 0.3


In [ ]:
# testing the ANPD model on video

!./darknet detector test /content/yolov4/data/obj.data /content/yolov4/data/obj.cfg /content/yolov4/outputs/yolov4-obj_last.weights /content/yolov4/test/vid1.mp4 -thresh 0.3
